In [ ]:
import numpy as np
from scipy import optimize
!pip install -q git+https://github.com/ott-jax/ott@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 15.5 MB/s eta 0:00:00


In [ ]:
import jax
import jax.numpy as jnp
from ott.geometry.pointcloud import PointCloud
from ott.tools.plot import Plot
from ott.solvers.linear import sinkhorn
from collections import namedtuple
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython import display

In [ ]:
outs = []
def plot_match_anim(n: int, frames: int=20):
  dim = 2
  fig = plt.figure(figsize=(8, 5))
  rngs = jax.random.split(jax.random.PRNGKey(0),3)
  x0 = jax.random.uniform(rngs[0],(n,dim))
  y = jax.random.uniform(rngs[1],(n,dim))
  x1 = jax.random.uniform(rngs[2],(n,dim))
  for t in jnp.linspace(0,1,frames):
    geom = PointCloud((1-t)*x0 + t*x1,y, epsilon=0.01)
    matrix = jax.jit(sinkhorn.solve)(geom).matrix
    OT = namedtuple('OT', 'matrix geom a b')
    outs.append(OT(matrix, geom, jnp.ones((n,))/n, jnp.ones((n,))/n))
  plott = Plot(fig=fig, threshold=1e-9)
  anim = plott.animate(outs, frame_rate=4)
  html = display.HTML(anim.to_jshtml())
  display.display(html)
  writervideo = animation.FFMpegWriter(fps=10)
  anim.save('smooth_sinkhorn.mp4', writer=writervideo)
  plt.close()


In [ ]:
plot_match_anim(20,200)